# Poblacion Argentina
Este cuaderno de Jupyter nos permite extraer información de dos fuentes diferentes para analizar homicidios en accidentes de tráfico:
1. API de Buenos Aires BA DATA:
Utilizaremos esta API para obtener información geográfica (barrio, comuna, comisaría, etc.) a partir de coordenadas de latitud y longitud. Esto nos ayudará a enriquecer nuestro conjunto de datos de homicidios en accidentes de tráfico con información contextual.
2. Censos Poblacional de 2010 a 2022 en Wikipedia:
Extraeremos datos de los censos para calcular indicadores clave de rendimiento (KPIs) relevantes para el proyecto.
**En resumen, combinaremos datos de ubicación con datos demográficos para obtener un análisis más completo de los homicidios en accidentes de tráfico.**

# Importaciones de Librerias


In [39]:
# importaciones
import pandas as pd
import requests
from bs4 import BeautifulSoup
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [40]:
homicidios = pd.read_csv('homicidios_hechos.csv')

In [41]:
homicidios.head()

,Id Hecho,Nro de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Dirección normalizada,Comuna,XY (CABA),POS X,POS Y,Participantes,Víctima,Acusado,Día de la Semana,Tipo de Día
0,2016-0001,1,2016-01-01,2016,Enero,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,Viernes,Día de Semana
1,2016-0002,1,2016-01-02,2016,Enero,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Fin de Semana
2,2016-0003,1,2016-01-03,2016,Enero,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,Domingo,Fin de Semana
3,2016-0004,1,2016-01-10,2016,Enero,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,Domingo,Fin de Semana
4,2016-0005,1,2016-01-21,2016,Enero,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,Día de Semana


## Desarrollo de una funcion de geolocalización:
##### Se ha implementado una función que determina el barrio de la Ciudad Autónoma de Buenos Aires (CABA) al que pertenece un punto específico, utilizando las coordenadas de latitud y longitud disponibles en el conjunto de datos (DataFrame).

In [42]:
def obtener_barrio(latitud, longitud):
    """
    Obtiene el nombre del barrio correspondiente a las coordenadas geográficas dadas.

    Parameters:
    -----------
        latitud (float): La latitud de la ubicación.
        longitud (float): La longitud de la ubicación.

    Returns:
    --------
        str: El nombre del barrio si se encuentra, o "Desconocido" si no se encuentra o hay un error.
    """
    url = "https://datosabiertos-usig-apis.buenosaires.gob.ar/datos_utiles"
    params = {
        "x": longitud,
        "y": latitud,
        "formato": "json"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
        if "barrio" in data:
            return data["barrio"]
        else:
            return "Desconocido"
    except requests.exceptions.RequestException as e:
        print("Error al hacer la solicitud:", e)
        return "Desconocido"

# Aplicación de la función
Aplicamos la función creada en una nueva columna del DataFrame Homicidios

homicidios['Barrio'] = homicidios.apply(lambda row: obtener_barrio(row['POS Y'], row['POS X']), axis=1)

In [43]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def obtener_barrio_coords(coords):
    latitud, longitud = coords
    url = "https://datosabiertos-usig-apis.buenosaires.gob.ar/datos_utiles"
    params = {
        "x": longitud,
        "y": latitud,
        "formato": "json"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Esto lanzará una excepción si la respuesta no es exitosa
        data = response.json()
        return data.get("barrio", "Desconocido")
    except Exception as e:
        print(f"Error al hacer la solicitud para {coords}: {e}")
        return "Desconocido"

def aplicar_obtener_barrio_concurrentemente(df, lat_col, long_col):
    coords = [(row[lat_col], row[long_col]) for index, row in df.iterrows()]
    results = []

    with ThreadPoolExecutor() as executor:
        future_to_coords = {executor.submit(obtener_barrio_coords, coord): coord for coord in coords}
        for future in as_completed(future_to_coords):
            results.append(future.result())
            
    return results

# Ejemplo de cómo usarlo con tu DataFrame
homicidios['Barrio'] = aplicar_obtener_barrio_concurrentemente(homicidios, 'POS Y', 'POS X')
homicidios.head()


Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)
Error al hacer la solicitud para (nan, nan): Expecting value: line 1 column 1 (char 0)


,Id Hecho,Nro de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Comuna,XY (CABA),POS X,POS Y,Participantes,Víctima,Acusado,Día de la Semana,Tipo de Día,Barrio
0,2016-0001,1,2016-01-01,2016,Enero,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,Viernes,Día de Semana,San Nicolas
1,2016-0002,1,2016-01-02,2016,Enero,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Fin de Semana,Villa Devoto
2,2016-0003,1,2016-01-03,2016,Enero,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,Domingo,Fin de Semana,Constitucion
3,2016-0004,1,2016-01-10,2016,Enero,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,Domingo,Fin de Semana,Barracas
4,2016-0005,1,2016-01-21,2016,Enero,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,Día de Semana,Parque Chacabuco


In [44]:
homicidios[['Barrio']]

,Barrio
0,San Nicolas
1,Villa Devoto
2,Constitucion
3,Barracas
4,Parque Chacabuco
...,...
689,Villa Santa Rita
690,Parque Chacabuco
691,Parque Avellaneda
692,Parque Avellaneda


##### Observamoos los homicidios que no tienen Barrios de residencia.

In [45]:
homicidios[homicidios.Barrio == '']


,Id Hecho,Nro de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Comuna,XY (CABA),POS X,POS Y,Participantes,Víctima,Acusado,Día de la Semana,Tipo de Día,Barrio
12,2016-0017,1,2016-02-15,2016,Febrero,15,16:54:00,16.0,MARIANO ACOSTA Y ANA MARIA JANER,AVENIDA,...,8,Point (101228.61094416 96623.72136186),-58.449897,-34.659704,PEATON-MOTO,PEATON,MOTO,Lunes,Día de Semana,
15,2016-0021,1,2016-02-17,2016,Febrero,17,23:35:00,23.0,AV DIRECTORIO Y CURAPALIGUE,AVENIDA,...,7,Point (101074.94198885 99848.19369046),-58.451577,-34.630638,AUTO-AUTO,AUTO,AUTO,Miércoles,Día de Semana,
16,2016-0022,1,2016-02-21,2016,Febrero,21,06:00:00,6.0,AV 9 DE JULIO Y ADOLFO ALSINA,AVENIDA,...,1,Point (107507.39502374 102046.08333077),-58.381447,-34.610799,MOTO-AUTO,MOTO,AUTO,Domingo,Fin de Semana,
23,2016-0034,1,2016-03-12,2016,Marzo,12,12:30:00,12.0,AV AMANCIO ALCORTA Y BONAVENA,AVENIDA,...,4,Point (105230.98315750 97613.97258373),-58.406239,-34.650765,PEATON-PASAJEROS,PEATON,PASAJEROS,Sábado,Fin de Semana,
28,2016-0039,1,2016-03-23,2016,Marzo,23,01:50:00,1.0,BARTOLOME MITRE Y AV PUEYRREDON,AVENIDA,...,3,Point (105252.60308563 102264.96028996),-58.406032,-34.608840,MOTO-PASAJEROS,MOTO,PASAJEROS,Miércoles,Día de Semana,
40,2016-0054,1,2016-04-23,2016,Abril,23,21:00:00,21.0,AU LUIS DELLEPIANE Y AV GRAL PAZ,AUTOPISTA,...,8,Point (97832.57143731 93489.83736533),-58.486954,-34.687952,MOTO-MOTO,MOTO,MOTO,Sábado,Fin de Semana,
49,2016-0069,1,2016-05-23,2016,Mayo,23,03:30:00,3.0,AV ENTRE RIOS Y PASAJE FILIBERTO,AVENIDA,...,3,Point (106612.54991859 100244.75667446),-58.391189,-34.627043,AUTO-CARGAS,AUTO,CARGAS,Lunes,Día de Semana,
73,2016-0098,1,2016-07-27,2016,Julio,27,16:08:00,16.0,AV SAN JUAN Y ALBERTI,AVENIDA,...,3,Point (105765.97394356 100631.47718024),-58.400424,-34.623562,MOTO-CARGAS,MOTO,CARGAS,Miércoles,Día de Semana,
77,2016-0102,1,2016-08-10,2016,Agosto,10,23:37:00,23.0,AV EVA PERON Y JOSE LEON SUAREZ,AVENIDA,...,9,Point (96625.80612216 95129.90776954),-58.500116,-34.673165,MOTO-CARGAS,MOTO,CARGAS,Miércoles,Día de Semana,
129,2016-0163,1,2016-12-10,2016,Diciembre,10,07:30:00,7.0,AV GRAL PAZ Y AV GENERAL MOSCONI,GRAL PAZ,...,11,Point (94486.28631712 103468.59928582),-58.523406,-34.597988,MOTO-AUTO,MOTO,AUTO,Sábado,Fin de Semana,


Ubicamos en el mapa esos siniestros sin Barrio

In [46]:
homicidios_sin_barrio = homicidios[homicidios['Barrio'] == '']

fig = px.scatter_mapbox(
    homicidios_sin_barrio,
    lat = 'POS Y',
    lon = 'POS X',
    zoom = 10,
    width = 800,   
    height = 680 
)

fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(title_x=0.5,
                  title_y=0.95, 
                  title_text='Siniestros sin datos de Barrio',
                  title_font=dict(size=24))
fig.update_traces(marker=dict(size=7, color='#113946'))  


fig.show()

Se imputan esos puntos poniendo los valores de esos Barrios vacios Sobre la General Paz como se observa en el mapa

In [47]:
homicidios['Barrio'] = homicidios['Barrio'].replace('', 'Sobre Gral. Paz')

#### Guardando el Dataframe

In [48]:
homicidios.to_csv('homicidios_hechos.csv', index=False, encoding='utf-8')

Scrapping  de Wikipedia
*La página que usaremos es esta: https://es.wikipedia.org/wiki/Censo_argentino_de_2022, el cual tiene información estadístico del Censo del 2010 y 2022 categorizado por Jurisdicción y Aglomerado Urbano.*

In [49]:
url = 'https://es.wikipedia.org/wiki/Censo_argentino_de_2022'

# Realizamos una solicitud GET a la página
response = requests.get(url)

# Analizamos el contenido HTML de la página
soup = BeautifulSoup(response.content, 'html.parser')

# Buscamos la segunda tabla de la pagina con la clase 'wikitable'
tabla = soup.find_all('table', class_='wikitable')[0]

# Extraemos los datos de la tabla y lo guardamos en una lista
datos = []
filas = tabla.find_all('tr')

for fila in filas[1:]:  
    celdas = fila.find_all(['th', 'td'])
    fila_datos = [celda.get_text(strip=True) for celda in celdas]
    datos.append(fila_datos)

Vemos los datos extraidos y seleccionamos los datos de la Ciudad Autónoma de Buenos Aires.

In [50]:
for fila in datos:
    print(fila)

['1', 'Provincia de Buenos Aires', '15.625.084[6]\u200b', '17.523.996[7]\u200b', '12,1', '9,8']
['2', 'Córdoba', '3.308.876', '3.840.905', '16,1', '15,5']
['3', 'Santa Fe', '3.194.537', '3.544.908', '11,0', '9,0']
['4', 'Ciudad Autónoma de Buenos Aires', '2.890.151', '3.121.707', '8,0', '6,4']
['5', 'Mendoza', '1.738.929', '2.043.540', '17,5', '12,3']
['6', 'Tucumán', '1.448.188', '1.731.820', '19,6', '13,6']
['7', 'Salta', '1.214.441', '1.441.351', '18,7', '14,3']
['8', 'Entre Ríos', '1.235.994', '1.425.578', '15,3', '12,0']
['9', 'Misiones', '1.101.593', '1.278.873', '16,1', '12,7']
['10', 'Corrientes', '992.595', '1.212.696', '22,2', '15,8']
['11', 'Chaco', '1.055.259', '1.129.606', '7,0', '6,7']
['12', 'Santiago del Estero', '874.006', '1.060.906', '21,4', '15,7']
['13', 'San Juan', '681.055', '822.853', '20,8', '15,4']
['14', 'Jujuy', '673.307', '811.611', '20,5', '14,3']
['15', 'Río Negro', '638.645', '750.768', '17,5', '14,8']
['16', 'Neuquén', '551.266', '710.814', '28,9', '23,

In [51]:
data_caba = datos[3]
data_caba

['4',
 'Ciudad Autónoma de Buenos Aires',
 '2.890.151',
 '3.121.707',
 '8,0',
 '6,4']

Seleccionamos los valores de la población del 2010 y 2022 de la lista

In [52]:

poblacion_2010 = int(data_caba[2].replace('.', '').replace(',', ''))
poblacion_2022 = int(data_caba[3].replace('.', '').replace(',', ''))

In [53]:
anios = list(range(2010, 2023))

poblacion_por_anio = [poblacion_2010 + (poblacion_2022 - poblacion_2010) * (anio - 2010) / 12 for anio in anios]

poblacion_por_anio = list(map(int, poblacion_por_anio))

In [54]:
poblacion_por_anio

[2890151,
 2909447,
 2928743,
 2948040,
 2967336,
 2986632,
 3005929,
 3025225,
 3044521,
 3063818,
 3083114,
 3102410,
 3121707]

Creamos un DataFrame con los datos

In [55]:
data = {'Año': anios, 'Población': poblacion_por_anio}
poblacion_caba = pd.DataFrame(data)

In [56]:
poblacion_caba

,Año,Población
0,2010,2890151
1,2011,2909447
2,2012,2928743
3,2013,2948040
4,2014,2967336
5,2015,2986632
6,2016,3005929
7,2017,3025225
8,2018,3044521
9,2019,3063818


Guardamos el DataFrame

In [57]:
poblacion_caba.to_csv('población_caba.csv', index= False, encoding='utf-8', sep=',')